In [ ]:
!pip install google-search-results

In [ ]:
!pip install langchain_community

In [ ]:
import os
import smtplib
import sqlite3
from email.mime.text import MIMEText
from twilio.rest import Client
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper

In [ ]:
# Set up API keys (replace with your actual API keys)
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
os.environ["SERPAPI_API_KEY"] = "your_serpapi_api_key"
os.environ["TWILIO_ACCOUNT_SID"] = "your_twilio_account_sid"
os.environ["TWILIO_AUTH_TOKEN"] = "your_twilio_auth_token"

In [ ]:
# Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Web Search Tool
search = SerpAPIWrapper()

def web_search(query: str):
    """Fetches real-time search results."""
    return search.run(query)

# Define AI Agent Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define AI Tools
search_tool = Tool(name="Web Search", func=web_search, description="Searches the web for the latest information.")


In [ ]:
def summarize_text(text: str):
    """Summarizes long articles or text inputs."""
    return llm.predict(f"Summarize this: {text}")

summarize_tool = Tool(name="Summarizer", func=summarize_text, description="Summarizes articles and text inputs.")

# Initialize AI Agent
agent = initialize_agent(
    tools=[search_tool, summarize_tool],
    llm=llm,
    agent="zero-shot-react-description",
    memory=memory,
    verbose=True
)

In [ ]:
# Database Setup
db_conn = sqlite3.connect("ai_agent_data.db")
cursor = db_conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS ai_responses (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        query TEXT,
        response TEXT
    )
""")
db_conn.commit()

def store_response(query, response):
    """Stores AI responses in the database."""
    cursor.execute("INSERT INTO ai_responses (query, response) VALUES (?, ?)", (query, response))
    db_conn.commit()


In [ ]:
def send_email(subject, body, recipient_email):
    """Sends an email with the AI agent's response."""
    sender_email = "your_email@example.com"
    sender_password = "your_generated_app_password"  # Use App Password

    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient_email

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [ ]:
# WhatsApp Integration using Twilio
def send_whatsapp_message(body, recipient_whatsapp):
    """Sends AI responses via WhatsApp."""
    client = Client(os.environ["TWILIO_ACCOUNT_SID"], os.environ["TWILIO_AUTH_TOKEN"])

    from_whatsapp_number = "whatsapp:+your_twilio_whatsapp_number"
    to_whatsapp_number = f"whatsapp:{recipient_whatsapp}"

    try:
        message = client.messages.create(
            body=body,
            from_=from_whatsapp_number,
            to=to_whatsapp_number
        )
        print(f"WhatsApp message sent! Message SID: {message.sid}")
    except Exception as e:
        print(f"Failed to send WhatsApp message: {e}")


In [ ]:
# Run AI Agent
query = "Latest trends in generative AI"
response = agent.run(query)
print("Agentic AI Response:", response)

In [ ]:
# Store response in database
store_response(query, response)

# Send response via email
recipient_email = "recipient@example.com"
send_email("AI Research Update", response, recipient_email)

# Send response via WhatsApp
recipient_whatsapp = "+recipient_phone_number"  # e.g., "+1234567890"
send_whatsapp_message(response, recipient_whatsapp)

db_conn.close()
